### Desarrollo del codigo principal

##### Librerias

In [1]:
# Librerias prehechas
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
import numpy as np

In [2]:
# Llamado a los modelos de autoria propia
import cnn
import exploracion
import resnet50 as res

##### Descarga de los datos

In [3]:
dataset, info = tfds.load('deep_weeds', with_info=True, as_supervised=True)

##### Almacenamiento por separado de las imagenes y los labels

In [4]:
datos = dataset['train']

X_rgb = []
X_gray = []
y = []

for image, label in tfds.as_numpy(datos):
    tensor = tf.convert_to_tensor(image)
    gray_img = tf.image.rgb_to_grayscale(tensor)

    X_rgb.append(image)
    X_gray.append(gray_img.numpy())
    y.append(label)

X_rgb = np.array(X_rgb, dtype=np.float32)
X_rgbg = np.concatenate([X_rgb, X_gray], axis=-1)
y = np.array(y)

del X_rgb
del X_gray
del dataset

In [5]:
print('Shapes')
# print(f'X_rgb: {X_rgb.shape}')
print(f'X_rgbg: {X_rgbg.shape}')
print(f'y: {y.shape}')

Shapes
X_rgbg: (17509, 256, 256, 4)
y: (17509,)


##### Reduccion dimensional de las imagenes para optimizar la carga computacional

In [6]:
# # Reducitos tamaño para poder optimizar el uso de memoria
# X_rgb = tf.convert_to_tensor(X_rgb, dtype=tf.float32)
# X_rgb = tf.image.resize(X_rgb, [196, 196], method='area')
        
# # Normalizamos las imágenes al rango [0, 1]
# X_scaled_rgb = (X_rgb / 255.0).numpy()

# del X_rgb

In [7]:
# Reducitos tamaño para poder optimizar el uso de memoria
X_rgbg = tf.convert_to_tensor(X_rgbg, dtype=tf.float32)
X_rgbg = tf.image.resize(X_rgbg, [128, 128], method='area')
        
# Normalizamos las imágenes al rango [0, 1]
X_scaled_rgbg = (X_rgbg / 255.0).numpy()

del X_rgbg

In [8]:
# Aplicación de one-hot encoding a las etiquetas
y_onehot = np.array(to_categorical(y))

In [9]:
print('Shapes')
# print(f'X_rgb: {X_scaled_rgb.shape}')
print(f'X_gray: {X_scaled_rgbg.shape}')
print(f'y: {y.shape}')

Shapes
X_gray: (17509, 128, 128, 4)
y: (17509,)


##### Exploración de las imagenes

In [10]:
descripciones = {
    0: 'chinee_apple',
    1: 'lantana',
    2: 'parkinsonia',
    3: 'parthenium',
    4: 'prickly_acacia',
    5: 'rubber_vine',
    6: 'siam_weed',
    7: 'snake_weed',
    8: 'negative (sin maleza)'
}

In [11]:
# analisis_clase = exploracion.Conteo_por_etiquetas(y, descripciones)
# analisis_clase.mostrar()

In [12]:
# analisis_canal = exploracion.Visualizacion_por_canal(X_rgb, X_gray, y)

In [13]:
# analisis_canal.informacion_promedio_color()

In [14]:
# analisis_canal.distribucion_canal_etiqueta()

##### Aplicación de las CNN

In [15]:
# redConvolucional_rgb = cnn.Propuesta_rgb(X_scaled_rgb, y_onehot)
# redConvolucional_rgb.CrearModelo()
# redConvolucional_rgb.EvaluarModelo()

In [16]:
redConvolucional_rgbgray = cnn.Propuesta_gray(X_scaled_rgbg, y_onehot)
redConvolucional_rgbgray.CrearModelo()
redConvolucional_rgbgray.EvaluarModelo()

c:\Users\abiab\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\abiab\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/100
394/394 ━━━━━━━━━━━━━━━━━━━━ 209s 506ms/step - accuracy: 0.4570 - loss: 1.7538 - val_accuracy: 0.5132 - val_loss: 1.7057
Epoch 2/100
394/394 ━━━━━━━━━━━━━━━━━━━━ 186s 473ms/step - accuracy: 0.5419 - loss: 1.3655 - val_accuracy: 0.5460 - val_loss: 1.3791
Epoch 3/100
394/394 ━━━━━━━━━━━━━━━━━━━━ 200s 507ms/step - accuracy: 0.5511 - loss: 1.2907 - val_accuracy: 0.5746 - val_loss: 1.3009
Epoch 4/100
394/394 ━━━━━━━━━━━━━━━━━━━━ 188s 476ms/step - accuracy: 0.5730 - loss: 1.2331 - val_accuracy: 0.5767 - val_loss: 1.2283
Epoch 5/100
394/394 ━━━━━━━━━━━━━━━━━━━━ 222s 565ms/step - accuracy: 0.5741 - loss: 1.2182 - val_accuracy: 0.5967 - val_loss: 1.1949
Epoch 6/100
394/394 ━━━━━━━━━━━━━━━━━━━━ 216s 549ms/step - accuracy: 0.5897 - loss: 1.1603 - val_accuracy: 0.5782 - val_loss: 1.2325
Epoch 7/100
394/394 ━━━━━━━━━━━━━━━━━━━━ 213s 541ms/step - accuracy: 0.6040 - loss: 1.1456 - val_accuracy: 0.5874 - val_loss: 1.3109
Epoch 8/100
394/394 ━━━━━━━━━━━━━━━━━━━━ 212s 537ms/step - accuracy: 

##### Aplicación de aprendizaje por transferencia utilizando el modelo ResNet50 como base